In [2]:
import pandas as pd

# AERONEFS

In [3]:
dfa = pd.read_csv('data/aeronefs_2023-11-07.csv')

display(dfa.shape)
display(dfa.head())

(232, 6)

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,B747_0195,B747,2008-12-15,2023-11-05,False,NaN
1,A320_4466,A320,2017-08-18,2023-11-04,False,NaN
2,None_3415,NaN,2007-11-08,2023-11-02,False,NaN
3,CRJ900_6891,CRJ900,2000-04-30,2023-10-30,False,NaN
4,A321_2784,A321,2014-02-18,2023-11-04,False,NaN


# COMPOSANTS

In [4]:
dfc = pd.read_csv('data/composants_2023-11-07.csv')

display(dfc.shape)
display(dfc.head())

(12257, 6)

,ref_compo,aero,desc,lifespan,taux_usure_actuel,cout_composant
0,TRA21-350,A380_3051,Train d'atterrissage avant,9162,14.77630,8599
1,EMP4-297,E175_5323,Empennage horizontal,12151,21.52365,5866
2,BOI22-504,B747_1933,Boîte noire (enregistreur de vol),10464,15.23360,9018
3,DIS15-232,E170_1621,Dispositifs de levage (flaps),10625,20.54875,6211
4,ORD12-146,E170_1621,Ordinateur de bord,8655,14.07325,5062


# DEGRADATION

In [14]:
# Créer un dossier 'deg' s'il n'existe pas en chemin relatif pour stocker les fichiers csv (à ne pas faire sur Colab)
import os
if not os.path.exists('data/deg'):
    os.makedirs('data/deg')
import shutil
if not os.path.exists('data/deg/degradations_2023-11-07.csv'):
    shutil.move('data/degradations_2023-11-07.csv', 'data/deg/degradations_2023-11-07.csv')

start_date = '2023-11-07'
today = pd.Timestamp.today().strftime('%Y-%m-%d')

date_range = pd.date_range(start=start_date, end=today, freq='D')
df_each_date = []
df_each_date.append(pd.read_csv('data/deg/degradations_2023-11-07.csv'))

for date in date_range:
    link = f'http://sc-e.fr/docs/degradations_{date.strftime("%Y-%m-%d")}.csv'
    
    try :
        df = pd.read_csv(link)
        df_each_date.append(df)
    
        export = os.path.join('data/deg', f'degradations_{date.strftime("%Y-%m-%d")}.csv')
        df.to_csv(export, index=False)
    except:
        print(f"Le fichier {link} n'existe pas")
        continue

dfd = pd.concat(df_each_date)

display(dfd.shape)
display(dfd.head())

Le fichier http://sc-e.fr/docs/degradations_2023-11-07.csv n'existe pas


(7315, 6)

,ref_deg,linked_aero,compo_concerned,usure_cumulée,measure_day,need_replacement
0,D009254,B767_0657,SPO16-014,21.70125,2023-11-07,False
1,D006897,B747_0910,DIS15-246,17.75015,2023-11-07,False
2,D007309,A350_1948,REA1-683,16.31745,2023-11-07,False
3,D000831,B747_4916,EQU23-377,19.24685,2023-11-07,False
4,D001545,E175_0964,TRA21-634,17.66480,2023-11-07,False


# LOGS VOLS

In [15]:
# Créer un dossier 'deg' s'il n'existe pas en chemin relatif pour stocker les fichiers csv (à ne pas faire sur Colab)
import os
if not os.path.exists('data/logs_vols'):
    os.makedirs('data/logs_vols')
import shutil
if not os.path.exists('data/logs_vols/logs_vols_2023-11-07.csv'):
    shutil.move('data/logs_vols_2023-11-07.csv', 'data/deg/logs_vols_2023-11-07.csv')

start_date = '2023-11-07'
today = pd.Timestamp.today().strftime('%Y-%m-%d')

date_range = pd.date_range(start=start_date, end=today, freq='D')
df_each_date = []
df_each_date.append(pd.read_csv('data/deg/degradations_2023-11-07.csv'))

for date in date_range:
    link = f'http://sc-e.fr/docs/logs_vols_{date.strftime("%Y-%m-%d")}.csv'
    
    try :
        df = pd.read_csv(link)
        df_each_date.append(df)
    
        export = os.path.join('data/logs_vols', f'logs_vols_{date.strftime("%Y-%m-%d")}.csv')
        df.to_csv(export, index=False)
    except:
        print(f"Le fichier {link} n'existe pas")
        continue

dfd = pd.concat(df_each_date)

display(dfd.shape)
display(dfd.head())

Le fichier http://sc-e.fr/docs/logs_vols_2023-11-07.csv n'existe pas


(3607, 12)

,ref_deg,linked_aero,compo_concerned,usure_cumulée,measure_day,need_replacement,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,D009254,B767_0657,SPO16-014,21.70125,2023-11-07,False,NaN,NaN,NaN,NaN,NaN,NaN
1,D006897,B747_0910,DIS15-246,17.75015,2023-11-07,False,NaN,NaN,NaN,NaN,NaN,NaN
2,D007309,A350_1948,REA1-683,16.31745,2023-11-07,False,NaN,NaN,NaN,NaN,NaN,NaN
3,D000831,B747_4916,EQU23-377,19.24685,2023-11-07,False,NaN,NaN,NaN,NaN,NaN,NaN
4,D001545,E175_0964,TRA21-634,17.66480,2023-11-07,False,NaN,NaN,NaN,NaN,NaN,NaN


# ------------------ test sur A320_2348 ------------------

In [6]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A320_2348 = dfa[dfa['aero'] == "A320_2348"]
dfc_A320_2348 = dfc[dfc['aero'] == "A320_2348"]
dfd_A320_2348 = dfd[dfd['aero'] == "A320_2348"]
dflv_A320_2348 = dflv[dflv['aero'] == "A320_2348"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A320_2348 = pd.merge(dfc_A320_2348, dfd_A320_2348, on=common_columns)
df_A320_2348 = pd.merge(df_A320_2348, dfa_A320_2348, on='aero')
df_A320_2348 = pd.merge(df_A320_2348, dflv_A320_2348, on='aero')

# ------------------ test sur A320_5619 ------------------

In [7]:

# Renommer les colonnes pour avoir le même nom de colonne dans chaque df
dataframes = [dfa, dfc, dfd, dflv]

for df in dataframes:
    if 'aero' not in df.columns:
        if 'ref_aero' in df.columns:
            df.rename(columns={'ref_aero': 'aero'}, inplace=True)
        elif 'linked_aero' in df.columns:
            df.rename(columns={'linked_aero': 'aero'}, inplace=True)
        elif 'aero_linked' in df.columns:
            df.rename(columns={'aero_linked': 'aero'}, inplace=True)
    if 'compo' not in df.columns:
        if 'ref_compo' in df.columns:
            df.rename(columns={'ref_compo': 'compo'}, inplace=True)
        elif 'compo_concerned' in df.columns:
            df.rename(columns={'compo_concerned': 'compo'}, inplace=True)

# Garder les lignes qui nous intéressent dans chaque df
dfa_A320_5619 = dfa[dfa['aero'] == "A320_5619"]
dfc_A320_5619 = dfc[dfc['aero'] == "A320_5619"]
dfd_A320_5619 = dfd[dfd['aero'] == "A320_5619"]
dflv_A320_5619 = dflv[dflv['aero'] == "A320_5619"]

common_columns = ['aero','compo']
# Merge les df à partir des colonnes communes
df_A320_5619 = pd.merge(dfc_A320_5619, dfd_A320_5619, on=common_columns)
df_A320_5619 = pd.merge(df_A320_5619, dfa_A320_5619, on='aero')
df_A320_5619 = pd.merge(df_A320_5619, dflv_A320_5619, on='aero')